In [ ]:
import os

import pandas as pd
import numpy as np

import sys

from smrt import make_snowpack, make_model, sensor_list, make_soil, SMRTError
from smrt.inputs.make_medium import make_medium

%load_ext autoreload
%autoreload 2

%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
sp_xls = pd.ExcelFile("data/CanadaData/SP.xlsx")

obs = pd.read_csv("data/CanadaData/TB.csv")

# each sheet is one snowpit
sps = [sp for sp in sp_xls.sheet_names]
len(sps)

In [ ]:

def prepare_snowpit(sp):

    profile = sp_xls.parse(sp)

    temp_substrate = min(profile['TempSol'].iloc[0], 273.15)

    profile['z'] = profile['Hneige'] * 0.01

    profile = profile.drop(columns=['TempSol', 'TEMPERATURE', 'IRIS', 'Hneige', 'Hneige_cm', 'Ropt', 'Hneige (m)']).dropna()

    profile = profile.rename(columns={'Temp_Kel': 'temperature',
                                      'Density': 'density'})

    return temp_substrate, profile
        
snowpits = {sp: prepare_snowpit(sp) for sp in sps}

#database_with_result['depth'] = [snowpits[sp][1]['z'].abs().max() for sp in database_with_result.index]

In [ ]:
freqs = ['19', '37']


def run_model(temp_substrate, profile, microstructure, coef):
    density_ice = 917.
    porod_length = 4 * (1 - profile.density / density_ice) / (density_ice * profile.SSA)
    
    K_TS = np.where(profile.SSA > 15, 0.6, coef)
    K_SEXP = np.where(profile.SSA > 15, 0.625, coef)
    K_SHS = np.where(profile.SSA > 15, 0.64, coef)
        
    microstructure_args = {
        "USEXP": dict(microstructure_model="unified_scaled_exponential", porod_length=porod_length, polydispersity=K_SEXP),
        "UTS": dict(microstructure_model="unified_teubner_strey", porod_length=porod_length, polydispersity=K_TS),
        "USHS": dict(microstructure_model="unified_sticky_hard_spheres", porod_length=porod_length, polydispersity=K_SHS)
    }

    substrate = make_soil("soil_wegmuller", permittivity_model="montpetit2008",
                          temperature=temp_substrate, roughness_rms=1.93e-3)

    sp = {m: make_medium(profile, **microstructure_args[m], substrate=substrate) for m in microstructure}

    model = make_model("iba", "dort", rtsolver_options=dict(prune_deep_snowpack=8, error_handling='nan'))
    sensor = sensor_list.amsr2(freqs)

    results = model.run(sensor, sp, parallel_computation=True)

    results = results.to_dataframe()
    results = results.unstack()
    results.index = ["_".join(ind) for ind in results.index]  # collapse multiindex by joining names with _

    return results


def run_all_site(microstructure, coef):
    results = []
    for sp in snowpits:
        if snowpits[sp] is None:
            continue
        res = run_model(*snowpits[sp], microstructure, coef)
        if res is None:
            continue
        results.append(res)

    results = pd.DataFrame(results, index=snowpits.keys())
    database_with_result = results.join(obs.set_index("CorresSP"))

    return database_with_result

In [ ]:

def compute_error(microstructure, coef_list):

    errors = []
    for coef in coef_list:
        database_with_result = run_all_site(microstructure=[microstructure], coef=coef)
        #print(database_with_result)
        d = {'polydispersity': coef}

        for freq in freqs:
            d[f'rmse_{freq}'] = np.sqrt(((database_with_result[f'{freq}V MES'] - database_with_result[f'{freq}V_{microstructure}'])**2).mean())
            d[f'abse_{freq}'] = (database_with_result[f'{freq}V MES'] - database_with_result[f'{freq}V_{microstructure}']).abs().mean()
            d[f'bias_{freq}'] = (database_with_result[f'{freq}V MES'] - database_with_result[f'{freq}V_{microstructure}']).mean()
            d[f'count_{freq}'] = database_with_result[f'{freq}V_{microstructure}'].dropna().count()
        errors.append(d)
    return pd.DataFrame(errors).set_index('polydispersity')

polydispersity = np.arange(1, 3, 0.05)
# polydispersity = np.arange(1.3, 1.4, 0.01)  # refine

lazzy = True
update = False

errors = {}
for m in ['UTS', 'USEXP', 'USHS']:
    
    filename = f"results/simulations-errors-canada-{m.lower()}.csv"
    file_exists = os.path.exists(filename)
    
    if (lazzy or update) and file_exists:
        errors[m] = pd.read_csv(filename).set_index('polydispersity').drop_duplicates()
    
    if update or not file_exists:
        err = compute_error(m, polydispersity)
        if update and file_exists:
            err = pd.concat((err, errors[m])).sort_index().drop_duplicates()
        errors[m] = err

        errors[m].to_csv(filename)
        
    errors[m]['rmse'] = np.sqrt((errors[m]['rmse_19']**2 + errors[m]['rmse_37']**2) / 2)
    errors[m]['bias'] = (errors[m]['bias_19'] + errors[m]['bias_37']) / 2
    
    imin = errors[m]['rmse'].argmin()
    #print(m, errors[m].index[imin])
    

In [ ]:
for m in errors:
    Koptimal = errors[m]['rmse'].idxmin()
    print(m, Koptimal, errors[m].loc[Koptimal]['rmse'])

In [ ]:
f, axs = plt.subplots(1, 3, figsize=(6, 3), sharey=True, sharex=True)

axs[2].plot(errors['UTS'].index, errors['UTS']['rmse'], '-',)
axs[2].plot(errors['UTS'].index, abs(errors['UTS']['bias']), '-',)
#axs[2].set_xlabel('Repeat coefficient $q$')  # = d / 2\\pi \\xi$')
axs[2].set_ylabel('RMSE 19 and 37 GHz, V-pol (K)')
#axs[2].set_ylabel('Error (K)')
#axs[2].set_ylim((0, 35))
axs[2].set_title('Teubner-Strey')

axs[0].plot(errors['USEXP'].index, errors['USEXP']['rmse'], '-', label='RMSE')
axs[0].plot(errors['USEXP'].index, abs(errors['USEXP']['bias']), '-', label='|bias|')
axs[0].set_title('Scaled exponential')
#axs[0].set_xlabel('Scaling coefficient $\\phi$')
axs[0].legend()

axs[1].plot(errors['USHS'].index, errors['USHS']['rmse'], '-')
axs[1].plot(errors['USHS'].index, abs(errors['USHS']['bias']), '-')

#axs[2].set_xlabel('Stickiness $\\tau$')
axs[1].set_title('Sticky Hard Sphere')

for i in [0, 1, 2]:
    axs[i].grid(alpha=0.2)
    axs[i].set_xlabel('Polydispersity $K$')

axs[2].set_xlim((1, 2.3))
f.tight_layout()
plt.savefig("fig-global-optimisation-coefs-arctic.pdf")